# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'DiamondDataHyperdrive'

experiment=Experiment(ws, experiment_name)

found = False
key = "diamond-data"
description_text = "Dataset to predict the price of round cut polished diamonds"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        print("Dataset not found, please register dataset in the workspace")


from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "aml-compute"
compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
print('Found existing cluster, use it.')

Found existing cluster, use it.


In [3]:
df = dataset.to_pandas_dataframe()
df.describe()
print(df)

from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(df, test_size=0.2, random_state=223)


      Column2  carat        cut color clarity  depth  table  price     x  \
0           1   0.23      Ideal     E     SI2   61.5   55.0    326  3.95   
1           2   0.21    Premium     E     SI1   59.8   61.0    326  3.89   
2           3   0.23       Good     E     VS1   56.9   65.0    327  4.05   
3           4   0.29    Premium     I     VS2   62.4   58.0    334  4.20   
4           5   0.31       Good     J     SI2   63.3   58.0    335  4.34   
...       ...    ...        ...   ...     ...    ...    ...    ...   ...   
53938   53939   0.86    Premium     H     SI2   61.0   58.0   2757  6.15   
53939   53940   0.75      Ideal     D     SI2   62.2   55.0   2757  5.83   
53940   53941   0.71    Premium     E     SI1   60.5   55.0   2756  5.79   
53941   53942   0.71    Premium     F     SI1   59.8   62.0   2756  5.74   
53942   53943   0.70  Very Good     E     VS2   60.5   59.0   2757  5.71   

          y     z  
0      3.98  2.43  
1      3.84  2.31  
2      4.07  2.31  
3      

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [4]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1)


#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "--LR": uniform(0.001, 0.05),
    "--maxDepth":choice(8, 10, 15, 20, 30)
    }
)

#TODO: Create your estimator and hyperdrive config
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

estimator = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=compute_target,
                      environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(run_config=estimator,
                                     hyperparameter_sampling=param_sampling,
                                     policy=early_termination_policy,
                                     primary_metric_name='Mean Squared Error',
                                     primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
                                     max_total_runs=30,
                                     max_concurrent_runs=4)

In [22]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [23]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

KeyError: 'log_files'

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [24]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print(best_run)
print(best_run_metrics)

Run(Experiment: DiamondDataHyperdrive,
Id: HD_2d5fef5b-5793-47ff-92c2-f83fc1e1379c_5,
Type: azureml.scriptrun,
Status: Completed)
{'LR:': 0.002394121420437253, 'maxDepth:': 30, 'Mean Squared Error': 0.3601736928736402}


In [25]:
#TODO: Save the best model
model = best_run.register_model(model_name='HGBM_model',
                                              model_path='./outputs/log/HGBM.pkl')

Test the best model on the test data


In [5]:
testModel = ScriptRunConfig(source_directory='.',
                      script='testBestHyperdriveModel.py',
                      compute_target=compute_target,
                      environment=sklearn_env)
script_run = experiment.submit(testModel)


Submitting /mnt/batch/tasks/shared/LS_root/mounts/clusters/nano-proj2/code/Users/raeesah_mangera/CapstoneProject directory for run. The size of the directory >= 25 MB, so it can take a few minutes.


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

